In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nbeats_keras.model import NBeatsNet as NBeatsKeras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from nbeats_pytorch.model import NBeatsNet as NBeatsPytorch
from keras.optimizers import RMSprop, Adam
import time
from keras.models import load_model
#from target_data_electronic70_7 import target_X, target_y ,test_X, test_y
#from m4databasis21_7 import base_domain,zt_in,zt_out,M4Meta,inputsize,train_12,train_12_y
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from tensorflow.keras.losses import Loss
import tensorflow as tf
#from m4databasis35_7_70_7 import train_35,train_35_y,train_70,train_70_y
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Add, Concatenate,Flatten,Reshape
#import pandas as pd
data = 'ele'
target_X= pd.read_csv(f"../data/{data}_train_input_7.csv").iloc[:,1:].values.astype(np.float32) / 10000
target_y =pd.read_csv(f"../data/{data}_train_output_7.csv").iloc[:,1:].values.astype(np.float32) / 10000

X_train = target_X[:-round(target_X.shape[0]*0.2),:].astype(np.float32)
y_train = target_y[:-round(target_y.shape[0]*0.2)].astype(np.float32)

X_train_val= target_X[-round(target_X.shape[0]*0.2):,:].astype(np.float32)
y_train_val =target_y[-round(target_y.shape[0]*0.2):].astype(np.float32)


test_X= pd.read_csv(f"../data/{data}_val_input_7.csv").iloc[:,1:].values.astype(np.float32) / 10000
test_y =pd.read_csv(f"../data/{data}_val_output_7.csv").iloc[:,1:].values.astype(np.float32) / 10000

#X_train=target_X.astype(np.float32)
#y_train=target_y.astype(np.float32)
#y_train.astype(np.float32)
backcast_length = X_train.shape[1]
forecast_length = y_train.shape[1]

2024-10-16 20:45:41.228501: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-16 20:45:41.302340: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-16 20:45:41.302359: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-10-16 20:45:41.663451: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:
X_train.shape, y_train.shape, X_train_val.shape, y_train_val.shape



((578, 168), (578, 24), (145, 168), (145, 24))

# 함수

In [10]:
#################################################################################
# loss SMAPE
class SMAPE(Loss):
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 예측 값의 차원을 맞춤
       # y_pred=tf.clip_by_value(y_pred, 1e-10, tf.reduce_max(y_pred))
       # y_true = tf.clip_by_value(y_true, 1e-10, tf.reduce_max(y_true))
        
        numerator = 100 * tf.abs(y_true- y_pred )
        denominator =  (tf.abs(y_true ) + tf.abs(y_pred))/2
        smape =  numerator /  denominator #tf.clip_by_value(denominator, 1e-10, tf.reduce_max(denominator))
        return tf.reduce_mean(smape)

#################################################################################
# loss MASE
class MASE(Loss):
    def __init__(self, training_data, period, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = self.calculate_scale(training_data, period)
    def seasonal_diff(data, period):
        return data[period:] - data[:-period]

    def calculate_scale(self, training_data, period):
        # 주기 차분 계산
        diff = seasonal_diff(training_data, period)
        scale = np.mean(np.abs(diff))
        return scale
    
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 차원 맞추기
        error = tf.abs(y_true - y_pred)
        return tf.reduce_mean(error / self.scale)

def seasonal_diff(data, period):
    return data[period:] - data[:-period]

#################################################################################
#################################################################################
# 하이퍼파라미터 인자 설정
def hyperparameter():
    # 1 backcast
    # 2 forecast
    # 3 inputdim
    # 4 outputdim
    # 5 unit
    # 6 bacth size
    return X_train.shape[1],y_train.shape[1],1,1,256

#################################################################################
# nbeats + I모델 생성 함수
def bulid_model(backcast_,forecast_,input_dim,output_dim,unit):
    model= NBeatsKeras(backcast_length=backcast_, 
                       forecast_length=forecast_,
                       input_dim=input_dim,
                       output_dim=output_dim,
                       stack_types=(NBeatsKeras.TREND_BLOCK,
                                    NBeatsKeras.TREND_BLOCK,
                                    
                                    NBeatsKeras.TREND_BLOCK,
                                    NBeatsKeras.SEASONALITY_BLOCK,
                                    NBeatsKeras.SEASONALITY_BLOCK,
                                    NBeatsKeras.SEASONALITY_BLOCK)
                   ,nb_blocks_per_stack=1, thetas_dim=(1,2,3,24,12,6),
                   share_weights_in_stack=True, hidden_layer_units=unit)
    return model 
#################################################################################
# nbeats + G모델 생성 함수    
def bulid_model_G(backcast_,forecast_,input_dim,output_dim,unit):
    model= NBeatsKeras(backcast_length=backcast_, 
                       forecast_length=forecast_,
                       input_dim=input_dim,
                       output_dim=output_dim,
                       stack_types=(NBeatsKeras.GENERIC_BLOCK,NBeatsKeras.GENERIC_BLOCK)
                   ,nb_blocks_per_stack=5, thetas_dim=(24,24),
                   share_weights_in_stack=False, hidden_layer_units=unit)
    return model 
#################################################################################
# nbeats + I모델 부트스트랩 샘플링 배깅

def train_bagging_models(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model(backcast,forecast,in_dim,out_dim,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 0, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=0, 
                  callbacks=[early_stop],
                 validation_data = [X_train_val,y_train_val])
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys
#################################################################################
# nbeats + I모델 부트스트랩 샘플링 배깅

def train_bagging_models_G(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model_G(backcast,forecast,in_dim,out_dim,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 0, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=0, 
                  callbacks=[early_stop],
                 validation_data = [X_train_val,y_train_val])
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys

#################################################################################



#################################################################################
# 예측

def bagging_predict(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return np.median(predictions, axis=0)

def bagging_predict2(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return predictions

# 모형적합

In [3]:
model_num = 10

mase_models = train_bagging_models(model_num, MASE(y_train,24),2000,10,8,0.0001)
mape_models = train_bagging_models(model_num,'mape',2000,10,8,0.0001)
smape_models = train_bagging_models(model_num, SMAPE(),2000,10,8,0.0001)
mae_models = train_bagging_models(model_num,'mae',2000,10,8,0.0001)
mse_models = train_bagging_models(model_num, 'mse',2000,10,8,0.0001)

2024-10-16 20:45:53.307332: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-10-16 20:45:53.307378: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ymlee2-desktop): /proc/driver/nvidia/version does not exist
2024-10-16 20:45:53.307952: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'########################################################Model5
'########################################################Model6
'########################################################Model7
'########################################################Model8
'########################################################Model9
'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'#######################################

In [4]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
pred4,_=mae_models
pred5,_=mse_models

smape_predictions = bagging_predict2(pred1, X_train_val)
mase_predictions =  bagging_predict2(pred2, X_train_val)
mape_predictions =  bagging_predict2(pred3, X_train_val)
mae_predictions = bagging_predict2(pred4, X_train_val)
mse_predictions =  bagging_predict2(pred5, X_train_val)


concat_G = np.concatenate([smape_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
sMAPE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mape_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MAPE= np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mase_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MASE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mae_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MAE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mse_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MSE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)


MSE, MASE, MAE, MAPE, sMAPE


5/5 [==============================] - 0s 3ms/step


(0.29316, 0.27555, 0.28179, 0.28414, 0.27406)

In [31]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
pred4,_=mae_models
pred5,_=mse_models

smape_predictions2 = bagging_predict2(pred1, test_X)
smape_predictions2 = np.median(np.concatenate([smape_predictions2],axis=0),axis=0)

mase_predictions2 =bagging_predict2(pred2, test_X)
mase_predictions2 = np.median(np.concatenate([mase_predictions2],axis=0),axis=0)

mape_predictions2 =bagging_predict2(pred3, test_X)
mape_predictions2 = np.median(np.concatenate([mape_predictions2],axis=0),axis=0)

mae_predictions2 = bagging_predict2(pred4,test_X)
mae_predictions2 = np.median(np.concatenate([mae_predictions2],axis=0),axis=0)

mse_predictions2 =bagging_predict2(pred5,test_X)
mse_predictions2 = np.median(np.concatenate([mse_predictions2],axis=0),axis=0)


#concat_mase = np.concatenate([np.nan_to_num(np.array(mase_predictions2), nan=0)])
#fin_pred_mase = np.median(concat_mase,axis=1)

#concat_mape = np.concatenate([np.nan_to_num(np.array(mape_predictions2), nan=0)])
#fin_pred_mape = np.median(concat_mape,axis=1)

#concat_smape = np.concatenate([np.nan_to_num(np.array(smape_predictions2), nan=0)])
#fin_pred_smape = np.median(concat_smape,axis=1)

#concat_mae = np.concatenate([np.nan_to_num(np.array(mae_predictions2), nan=0)])
#fin_pred_mae = np.median(concat_mae,axis=1)

#concat_mse = np.concatenate([np.nan_to_num(np.array(mse_predictions2), nan=0)])
#fin_pred_mse = np.median(concat_mse,axis=1)

performance = np.array([MAE, MAPE,sMAPE,MSE,MASE])
beta = 3 # 조정 파라미터
weights = np.exp(-beta * performance)

gd= np.concatenate([mae_predictions2,
                    mape_predictions2,
                   smape_predictions2,
                   mse_predictions2,
                   mase_predictions2],axis=0)
#gd=np.median(gd,axis=2)
normalized_weights = weights / np.sum(weights)

# 각 모델의 예측값에 가중치를 부여하여 앙상블 예측 생성
ensemble_prediction = np.dot(normalized_weights, gd.reshape(5,-1))
print('############################################################################################') 
print('############################################################################################') 
pd.DataFrame(ensemble_prediction.flatten()).to_csv('exp7/NBEATs_I.csv')


print('exp',np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten())).round(5))

12/12 [==============================] - 0s 3ms/step
############################################################################################
############################################################################################
exp 0.29365


In [32]:
eee = []
for i in range(1,20):
    weights = np.exp(-i* performance)
    normalized_weights = weights / np.sum(weights)
    ensemble_prediction = np.dot(normalized_weights, gd.reshape(5,-1))
    eee.append((i,np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten().round(5)))))
    #print(f'exp_beta{i}',np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten())).round(5))

eee

[(1, 0.29329318),
 (2, 0.29357746),
 (3, 0.29365122),
 (4, 0.29367256),
 (5, 0.29368198),
 (6, 0.2936887),
 (7, 0.29369485),
 (8, 0.29370087),
 (9, 0.2937068),
 (10, 0.29371285),
 (11, 0.29371873),
 (12, 0.2937247),
 (13, 0.29373068),
 (14, 0.2937366),
 (15, 0.2937425),
 (16, 0.29374844),
 (17, 0.29375428),
 (18, 0.29376012),
 (19, 0.29376605)]

In [33]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
pred4,_=mae_models
pred5,_=mse_models

smape_predictions_G = bagging_predict2(pred1, test_X)
mase_predictions_G = bagging_predict2(pred2,test_X)
mape_predictions_G = bagging_predict2(pred3,test_X)
mae_predictions_G = bagging_predict2(pred4, test_X)
mse_predictions_G = bagging_predict2(pred5,test_X)


print('############################################################################################') 
print('############################################################################################') 

concat_G = np.concatenate([smape_predictions_G, mase_predictions_G,mape_predictions_G,mae_predictions_G,mse_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('all',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5)) 

concat_G = np.concatenate([smape_predictions_G, mase_predictions_G,mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('original',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mse_predictions_G, mase_predictions_G,mae_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('best',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mse_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mse',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))


concat_G = np.concatenate([mase_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mase',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mae_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mae',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mape',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([smape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('smape',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

12/12 [==============================] - 0s 2ms/step
############################################################################################
############################################################################################
all 0.29196
original 0.29126
best 0.29346
mse 0.30746
mase 0.29263
mae 0.29721
mape 0.29917
smape 0.29351


# 일반블락

In [11]:
model_num = 10

mase_models_G = train_bagging_models_G(model_num, MASE(y_train,24),2000,10,8,0.001)
mape_models_G = train_bagging_models_G(model_num,'mape',2000,10,8,0.001)
smape_models_G = train_bagging_models_G(model_num, SMAPE(),2000,10,8,0.001)
mae_models_G = train_bagging_models_G(model_num,'mae',2000,10,8,0.001)
mse_models_G = train_bagging_models_G(model_num, 'mse',2000,10,8,0.001)

'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'########################################################Model5
'########################################################Model6
'########################################################Model7
'########################################################Model8
'########################################################Model9
'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'#######################################

In [23]:
pred1,_=smape_models_G
pred2,_=mase_models_G
pred3,_=mape_models_G
pred4,_=mae_models_G
pred5,_=mse_models_G

smape_predictions_G = bagging_predict2(pred1, X_train_val)
mase_predictions_G = bagging_predict2(pred2,X_train_val)
mape_predictions_G = bagging_predict2(pred3, X_train_val)
mae_predictions_G = bagging_predict2(pred4, X_train_val)
mse_predictions_G = bagging_predict2(pred5, X_train_val)

concat_G = np.concatenate([smape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
sMAPE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MAPE= np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mase_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MASE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mae_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MAE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mse_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MSE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)


MSE, MASE, MAE, MAPE, sMAPE

5/5 [==============================] - 0s 2ms/step


(0.29786, 0.30226, 0.29426, 0.29467, 1.8118)

In [28]:
pred1,_=smape_models_G
pred2,_=mase_models_G
pred3,_=mape_models_G
pred4,_=mae_models_G
pred5,_=mse_models_G

smape_predictions2 = bagging_predict2(pred1, test_X)
smape_predictions2 = np.median(np.concatenate([smape_predictions2],axis=0),axis=0)

mase_predictions2 =bagging_predict2(pred2, test_X)
mase_predictions2 = np.median(np.concatenate([mase_predictions2],axis=0),axis=0)

mape_predictions2 =bagging_predict2(pred3, test_X)
mape_predictions2 = np.median(np.concatenate([mape_predictions2],axis=0),axis=0)

mae_predictions2 = bagging_predict2(pred4,test_X)
mae_predictions2 = np.median(np.concatenate([mae_predictions2],axis=0),axis=0)

mse_predictions2 =bagging_predict2(pred5,test_X)
mse_predictions2 = np.median(np.concatenate([mse_predictions2],axis=0),axis=0)


#concat_mase = np.concatenate([np.nan_to_num(np.array(mase_predictions2), nan=0)])
#fin_pred_mase = np.median(concat_mase,axis=1)

#concat_mape = np.concatenate([np.nan_to_num(np.array(mape_predictions2), nan=0)])
#fin_pred_mape = np.median(concat_mape,axis=1)

#concat_smape = np.concatenate([np.nan_to_num(np.array(smape_predictions2), nan=0)])
#fin_pred_smape = np.median(concat_smape,axis=1)

#concat_mae = np.concatenate([np.nan_to_num(np.array(mae_predictions2), nan=0)])
#fin_pred_mae = np.median(concat_mae,axis=1)

#concat_mse = np.concatenate([np.nan_to_num(np.array(mse_predictions2), nan=0)])
#fin_pred_mse = np.median(concat_mse,axis=1)

performance1 = np.array([MAE, MAPE,sMAPE,MSE,MASE])
beta = 3 # 조정 파라미터
weights = np.exp(-beta * performance)

gd= np.concatenate([mae_predictions2.reshape(1,-1),
                    mape_predictions2.reshape(1,-1),
                   smape_predictions2.reshape(1,-1),
                   mse_predictions2.reshape(1,-1),
                   mase_predictions2.reshape(1,-1)],axis=0)
#gd=np.median(gd,axis=2)
normalized_weights = weights / np.sum(weights)

# 각 모델의 예측값에 가중치를 부여하여 앙상블 예측 생성
ensemble_prediction = np.dot(normalized_weights,  gd.reshape(5,-1))

print('############################################################################################') 
print('############################################################################################') 

pd.DataFrame(ensemble_prediction.flatten()).to_csv('exp7/NBEATs_G.csv')
print('exp7',np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten())).round(5))

12/12 [==============================] - 0s 4ms/step
############################################################################################
############################################################################################
exp7 0.29721


In [29]:
eee = []
for i in range(1,20):
    weights = np.exp(-i* performance)
    normalized_weights = weights / np.sum(weights)
    ensemble_prediction = np.dot(normalized_weights, gd.reshape(5,-1))
    eee.append((i,np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten().round(5)))))
    #print(f'exp_beta{i}',np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten())).round(5))

eee

[(1, 0.30944183),
 (2, 0.29757154),
 (3, 0.2972139),
 (4, 0.29725108),
 (5, 0.2972566),
 (6, 0.29724938),
 (7, 0.29723904),
 (8, 0.29722807),
 (9, 0.29721698),
 (10, 0.29720584),
 (11, 0.29719487),
 (12, 0.29718387),
 (13, 0.29717293),
 (14, 0.2971621),
 (15, 0.29715127),
 (16, 0.29714045),
 (17, 0.29712966),
 (18, 0.29711896),
 (19, 0.29710832)]

In [16]:
pred1,_=smape_models_G
pred2,_=mase_models_G
pred3,_=mape_models_G
pred4,_=mae_models_G
pred5,_=mse_models_G

smape_predictions_G = bagging_predict2(pred1, test_X)
mase_predictions_G = bagging_predict2(pred2,test_X)
mape_predictions_G = bagging_predict2(pred3,test_X)
mae_predictions_G = bagging_predict2(pred4, test_X)
mse_predictions_G = bagging_predict2(pred5,test_X)
print('############################################################################################') 
print('############################################################################################') 

concat_G = np.concatenate([smape_predictions_G, mase_predictions_G,mape_predictions_G,mae_predictions_G,mse_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('all',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5)) 

concat_G = np.concatenate([smape_predictions_G, mase_predictions_G,mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('original',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mse_predictions_G, mase_predictions_G,mae_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('best',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mse_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mse',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))


concat_G = np.concatenate([mase_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mase',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mae_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mae',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mape',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([smape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('smape',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

12/12 [==============================] - 0s 3ms/step
############################################################################################
############################################################################################
all 0.29608
original 0.2947
best 0.30092
mse 0.30474
mase 0.30642
mae 0.30073
mape 0.29408
smape 1.79963
